In [1]:
import pandas as pd
import json
import geopandas as gpd
from shapely.geometry import Point
import requests
from collections import defaultdict

/Users/henrik_hao/opt/anaconda3/envs/MAST30034/lib/python3.10/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/var/folders/hl/0j63l0bd7n3ggf_3ylf9n8q80000gn/T/ipykernel_68735/472444752.py:3: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, 

In [8]:
# read into shape file
sa2_sf = gpd.read_file("../data/statistical_area_level2/SA2_2021_AUST_GDA2020.shp")
sa2_sf['geometry'] = sa2_sf['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
victoria_sa2 = sa2_sf[sa2_sf['STE_NAME21'] == 'Victoria']

In [6]:
lga_sf = gpd.read_file("../data/LGA_2023_AUST_GDA2020/LGA_2023_AUST_GDA2020.shp")
lga_sf['geometry'] = lga_sf['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
victoria_lga = lga_sf[lga_sf['STE_NAME21'] == 'Victoria']

In [10]:
victoria_sa2.head()

,SA2_CODE21,SA2_NAME21,CHG_FLAG21,CHG_LBL21,SA3_CODE21,SA3_NAME21,SA4_CODE21,SA4_NAME21,GCC_CODE21,GCC_NAME21,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,geometry
644,201011001,Alfredton,0,No change,20101,Ballarat,201,Ballarat,2RVIC,Rest of Vic.,2,Victoria,AUS,Australia,52.7109,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((143.78282 -37.56666, 143.75558 -37.5..."
645,201011002,Ballarat,0,No change,20101,Ballarat,201,Ballarat,2RVIC,Rest of Vic.,2,Victoria,AUS,Australia,12.3787,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((143.81896 -37.55582, 143.81644 -37.5..."
646,201011005,Buninyong,0,No change,20101,Ballarat,201,Ballarat,2RVIC,Rest of Vic.,2,Victoria,AUS,Australia,51.5855,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((143.84171 -37.61596, 143.84176 -37.6..."
647,201011006,Delacombe,0,No change,20101,Ballarat,201,Ballarat,2RVIC,Rest of Vic.,2,Victoria,AUS,Australia,34.1607,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((143.75050 -37.59119, 143.75044 -37.5..."
648,201011007,Smythes Creek,0,No change,20101,Ballarat,201,Ballarat,2RVIC,Rest of Vic.,2,Victoria,AUS,Australia,104.7274,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((143.73296 -37.62333, 143.73263 -37.6..."


In [11]:
victoria_lga.head()

,LGA_CODE23,LGA_NAME23,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM,LOCI_URI21,geometry
131,20110,Alpine,2,Victoria,AUS,Australia,4788.1570,https://linked.data.gov.au/dataset/asgsed3/LGA...,"POLYGON ((146.67058 -36.56827, 146.67056 -36.5..."
132,20260,Ararat,2,Victoria,AUS,Australia,4211.1172,https://linked.data.gov.au/dataset/asgsed3/LGA...,"POLYGON ((142.71709 -37.15729, 142.71705 -37.1..."
133,20570,Ballarat,2,Victoria,AUS,Australia,739.0321,https://linked.data.gov.au/dataset/asgsed3/LGA...,"POLYGON ((143.86292 -37.38007, 143.85880 -37.4..."
134,20660,Banyule,2,Victoria,AUS,Australia,62.5402,https://linked.data.gov.au/dataset/asgsed3/LGA...,"POLYGON ((145.08875 -37.69135, 145.08926 -37.6..."
135,20740,Bass Coast,2,Victoria,AUS,Australia,865.8094,https://linked.data.gov.au/dataset/asgsed3/LGA...,"MULTIPOLYGON (((145.10992 -38.51956, 145.10976..."


In [20]:
victoria_sa2.shape

(524, 17)

In [25]:
mapping = []
victoria_lga = victoria_lga.reset_index(drop=True)
victoria_sa2 = victoria_sa2.reset_index(drop=True)

# b. Iterate over each sa2 district:
for index, sa2_row in victoria_sa2.iterrows():
    # Check if the geometry is not None
    if sa2_row['geometry'] is not None:
        # Find the intersection area between the current sa2 district and all lga districts
        intersections = victoria_lga.intersection(sa2_row['geometry'])
        
        # Calculate the area of each intersection
        areas = intersections.area
        
        # Find the index of the lga district with the maximum intersection area
        max_area_index = areas.idxmax()
        
        # Check if max_area_index is not NaN and the maximum intersection area is greater than zero
        if pd.notna(max_area_index) and areas[max_area_index] > 0:
            # Append the mapping to the list
            mapping.append({
                'SA2_CODE21': sa2_row['SA2_CODE21'],
                'SA2_NAME21': sa2_row['SA2_NAME21'],  # Extracting the sa2 name
                'LGA_CODE23': victoria_lga.iloc[max_area_index]['LGA_CODE23'],
                'LGA_NAME23': victoria_lga.iloc[max_area_index]['LGA_NAME23']  # Extracting the lga name
            })
        else:
            # Handle the case where there's no valid intersection (e.g., log a warning or skip)
            print(f"Warning: No valid intersection found for SA2_CODE21: {sa2_row['SA2_CODE21']}")
    else:
        print(f"Warning: Missing geometry for SA2_CODE21: {sa2_row['SA2_CODE21']}")

# 3. Convert the Mapping List to a DataFrame
mapping_df = pd.DataFrame(mapping)



/var/folders/hl/0j63l0bd7n3ggf_3ylf9n8q80000gn/T/ipykernel_68735/1295910189.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  areas = intersections.area


In [26]:
mapping_df.to_csv("../data/sa2_to_lga.csv")